# Week 4 Geospatial Data with Folium
### Imports and Initializations

In [1]:
import numpy as np
import pandas as pd
import os
import folium
from folium import plugins
from folium.plugins import HeatMap, HeatMapWithTime

### Introduction to Folium Map

In [2]:
mapSF1 = folium.Map(
    location = [37.7749, -122.4194],
    tiles = 'Stamen Toner',
    zoom_start = 13)

folium.Marker([37.77919, -122.41914],
              popup='City Hall',
              icon=folium.Icon(color='blue',
                            icon='university',
                            prefix='fa')).add_to(mapSF1)

mapSF1

### Working on the SF Crime Dataset

In [3]:
# Pick the filename of the dataset to import in pandas
fileName = 'Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'
filePath = os.path.abspath(os.path.join(os.getcwd(), '..','Datasets', fileName))

# Raw Dataframe as created directly from csv
df_raw = pd.read_csv(filePath)

# Edit and store the Dataframe
df = df_raw.copy()
df['Date'] = pd.to_datetime(df['Date'])


### Plotting Crime Instances on Map

In [4]:
# Select Crime of Interest and Start/End Dates
crime = 'DRUG/NARCOTIC'
start_date = '2015-06-01'
end_date = '2015-06-02'

# Create a filtered dataframe based on new preferences
df1 = df.loc[(df['Date'] >= start_date)
            &(df['Date'] <= end_date)
            &(df['Category']==crime)].copy()

# Create New Map instance
mapSF1 = folium.Map(
    location = [37.7749, -122.4194],
    tiles = 'Stamen Toner',
    zoom_start = 12)

# Add Makers to the map (based on preferences from above)
for i, row in df1.iterrows():
    folium.CircleMarker([row['Y'], row['X']],
                        radius=1,
                        popup=row['Date'].date(),
                        color='red').add_to(mapSF1)

mapSF1

### Visualizing Crimes with Heatmaps

In [6]:
# Create a filtered dataframe based on new preferences
crime = 'SEX OFFENSES, NON FORCIBLE'
df2 = df[['X','Y']].loc[df['Category']==crime].copy()

# Create New Map Instance
mapSF2 = folium.Map(
    location = [37.7749, -122.4194],
    tiles = 'Stamen Toner',
    zoom_start = 13)

# Create Heat Data Coordinates List
heat_data = [[row['Y'],row['X']] for i, row in df2.iterrows()]

# Generate the Heatmap
HeatMap(heat_data).add_to(mapSF2)
mapSF2

### Visualizing Heatmaps with Time

In [7]:
# Pick Crime of Interest
crime = 'PROSTITUTION'

# Create filtered/edited dataframe
df3 = df[['Date','X','Y']].loc[df['Category']==crime].copy()
df3['Yr-Month'] = pd.to_datetime(df3['Date']).dt.to_period('M')

# Create the specified time interval list
monthsList = sorted(df3['Yr-Month'].unique())
monthsList = list(map(lambda x: str(x), monthsList))

# Pick Coordinates X,Y for each crime occurence for each specified time interval
heatMonthData = [[[row['Y'],row['X']] for index, row in
                  df3[df3['Yr-Month']==i].iterrows()] for i in
                 monthsList]

# Create New Map Instance
mapSF3 = folium.Map(
    location = [37.7749, -122.4194],
    tiles = 'Stamen Toner',
    zoom_start = 12)

# Create Time Heat Map
heatmap = HeatMapWithTime(data=heatMonthData,
                          radius=12,
                          auto_play=True,
                          max_opacity=0.7)
heatmap.add_to(mapSF3)

mapSF3